In [1]:
%load_ext cython

# A new "Vector" extension type

In [2]:
%%cython
from libc.stdlib cimport realloc, free 

cdef class Vector:
    cdef:
        double *vec          # Array of doubles
        long length          # Length
        public double[:] mv  # Memory view, can see from Python!

    def __cinit__(self, long length):
        self.change_length(length)
        
    cpdef change_length(self, long newlength):
        self.vec = <double*>realloc(self.vec, newlength * sizeof(double))
        self.mv = <double[:newlength]>self.vec
        self.length = newlength
        
    def __dealloc__(self):
        free(self.vec)  # <--- Memory leak if you don't do this 😱😱😱
        pass

## Helpful tool for showing memory use

In [3]:
# "psutil" is 3rd party. Install with "pip install psutil"
import os, psutil
def memory():
    pid = os.getpid()
    proc = psutil.Process(pid)
    mem = proc.memory_info()
    print('Memory used: {:.2f} MB'.format(mem.vms/1024/1024))
    
# Example call
memory()

Memory used: 2480.77 MB


# A small array

In [4]:
a = Vector(2)
memory()

Memory used: 2480.77 MB


# A large array

In [5]:
a = Vector(2**29)
memory()

Memory used: 6576.77 MB


Dynamically change vector size **down**

In [6]:
a.change_length(2)
memory()

Memory used: 2480.77 MB


Dynamically change vector size **up**

In [7]:
a.change_length(2**29)
memory()

Memory used: 6576.77 MB


# Reassign `a`? Memory should be stay the same

In [8]:
a = Vector(2**29)
memory()

Memory used: 6576.77 MB


# Delete "`a`"? Memory should go back down

In [9]:
del(a)
memory()

Memory used: 2480.77 MB
